In [14]:
%pylab inline

import os
from lammpsrun import LAMMPS, Prism
from PBSJobSubmissionScript import PBS_Submitter

from ase import Atoms, units
from ase.visualize import view
from ase.io.trajectory import Trajectory
from ase.io import write
from ase.build import molecule

cx1_out = r"/rds/general/user/yx6015/home/cx1_out/"
sim_path = r"/rds/general/user/yx6015/home/LAMMPS_Simulation/"
print(cx1_out + '\n' + sim_path)

Populating the interactive namespace from numpy and matplotlib
/rds/general/user/yx6015/home/cx1_out/
/rds/general/user/yx6015/home/LAMMPS_Simulation/


# Creating a grid to put water molecules

In [2]:
Atoms_grid = Atoms('Ne', pbc=True)
box_length = ((125 / (55.5 * cst.Avogadro)) * (1e+27))**(1 / 3)
Atoms_grid.set_cell((box_length, box_length, box_length))
Atoms_grid.center()
Atoms_grid = Atoms_grid.repeat((5, 5, 5))
Atoms_grid.rattle(stdev= box_length/5 - 1)

In [3]:
H2O_bulk = molecule('H2O')
H2O_bulk.euler_rotate(rand()*360, rand()*360, rand()*360, center="COM")
H2O_bulk.translate(Atoms_grid.get_positions()[0])
H2O_bulk.set_cell(Atoms_grid.get_cell())
H2O_bulk.set_pbc(True)

In [4]:
for new_pos in Atoms_grid.get_positions()[1:]:
    new_molecule = molecule('H2O')
    new_molecule.euler_rotate(rand()*360, rand()*360, rand()*360, center="COM")
    new_molecule.translate(new_pos)
    
    H2O_bulk += new_molecule.copy()
H2O_bulk.center()

# Equilibriate at 300 K

In [9]:
H2O_bulk_reaxff_params = {
    # Initialization
    "units": "real",
    "atom_style": "charge",
    "mass": ["1 1.008",
             "2 15.999"],
    "velocity": ["all create 300.0 1050027 rot yes dist gaussian"],
    
    # Forcefield definition
    "pair_style": "reax/c NULL safezone 16",
    "pair_coeff": ['* * ' + 'ffield.reax.Fe_O_C_H_combined' + ' H O'],
    "neighbor": "2.0 bin",
    #"neighbor_modify": "delay 10 check yes",
    
    # Run and Minimization
    "run": "100000",
    "timestep": 0.5,
    "fix": ["1 all nve",
            "2 all qeq/reax 1 0.0 10.0 1e-6 reax/c"]#,
            #"2 all temp/berendsen 300.0 300.0 100"]
}

In [10]:
H2O_bulk_calc = LAMMPS(parameters=H2O_bulk_reaxff_params,
                       no_data_file=True, always_triclinic=True)

H2O_bulk.set_calculator(H2O_bulk_calc)
H2O_bulk_calc.atoms = H2O_bulk.copy()
H2O_bulk_calc.prism = Prism(H2O_bulk_calc.atoms.get_cell())

In [11]:
H2O_bulk_calc.write_lammps_in(lammps_in=sim_path+"fe_aq/aq_125_300K.in",
                              lammps_trj="aq_125_300K.lammpstrj")

In [12]:
H2O_bulk_PBS = PBS_Submitter(job_names="H2O_bulk_equilibriation",
                             job_commands="mpiexec lmp_mpi -in {0} -screen none",
                             modules=["mpi", "lammps", "intel-suite/2017.6"],
                             walltime="4:00:00",
                             proc_nodes=1,
                             proc_cpus=4,
                             proc_mpiprocs=4,
                             memory=4)
H2O_bulk_PBS.params

{'job_commands': ['mpiexec lmp_mpi -in {0} -screen none'],
 'job_names': ['H2O_bulk_equilibriation'],
 'memory': [4],
 'modules': [['mpi', 'lammps', 'intel-suite/2017.6']],
 'proc_cpus': [4],
 'proc_mpiprocs': [4],
 'proc_nodes': [1],
 'walltime': ['4:00:00']}

In [16]:
curr_dir = os.getcwd()
os.chdir(sim_path + "fe_aq")

H2O_bulk_out, H2O_bulk_err = H2O_bulk_PBS.run()

os.chdir(curr_dir)

Submitted Job: H2O_bulk_equilibriation
b'2123480.cx1\n' b''


In [18]:
H2O_bulk_out

['2123480.cx1']